In [10]:
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
import numpy as np 
import seaborn as sns

pd.set_option('display.max_columns', None)



In [11]:
df = pd.read_csv(r'C:\Users\abhir\Projects\DS_Salary_Project\eda_data.csv')

# choose relevant columns 
df_model = df[['avg_salary','Rating','Size','Type of ownership','Industry','Sector','Revenue','num_competitors','hourly','employer_provided',
             'job_state','same_state','age','python_yn','spark','aws','excel','job_simple','seniority','desc_len']]

# get dummy data 
df_dum = pd.get_dummies(df_model)

In [12]:
X = df_dum.drop('avg_salary', axis =1)
y = df_dum.avg_salary.values

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Assuming X is your feature DataFrame
z_scores = np.abs((X - X.mean()) / X.std())
outliers = (z_scores > 3).any(axis=1)

# Get outlier rows
outlier_rows = X[outliers]
print("Outlier Rows:\n", outlier_rows)


Outlier Rows:
      Rating  num_competitors  hourly  employer_provided  same_state  age  \
0       3.8                1       0                  0           0   47   
1       3.4                1       0                  0           0   36   
2       4.8                1       0                  0           1   10   
3       3.8                3       0                  0           1   55   
4       2.9                3       0                  0           1   22   
..      ...              ...     ...                ...         ...  ...   
736     3.6                3       0                  0           0    3   
738     4.4                3       0                  0           0   14   
739     2.6                1       0                  0           1   36   
740     3.2                1       0                  0           0   -1   
741     3.6                1       0                  0           0   53   

     python_yn  spark  aws  excel  desc_len  Size_-1  Size_1 to 50 emplo